In [1]:
import os
import cv2

In [2]:
from keras.preprocessing import image


In [3]:
categories =  ['with_mask','without_mask']

In [4]:
data = []
for category in categories:
    path = os.path.join('train',category)
    label = categories.index(category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])
       
  

In [5]:
len(data)

2188

In [6]:
import random

In [7]:
random.shuffle(data)

In [8]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

In [9]:
len(X)

2188

In [10]:
import numpy as np

In [11]:
X = np.array(X)
y = np.array(y)

In [12]:
X.shape

(2188, 224, 224, 3)

In [13]:
y.shape

(2188,)

In [14]:
X = X/255

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [16]:
X_test.shape

(657, 224, 224, 3)

In [17]:
from keras.applications.vgg16 import VGG16

In [18]:
vgg = VGG16()

In [19]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [20]:
from keras import Sequential

In [21]:
model = Sequential()

In [22]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [24]:
for layer in model.layers:
    layer.trainable=False

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [26]:
from keras.layers import Dense

In [27]:
model.add(Dense(1,activation='sigmoid'))

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [29]:
 model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
48/48 [==============================] - 267s 6s/step - loss: 0.5932 - accuracy: 0.6852 - val_loss: 0.4246 - val_accuracy: 0.8661
Epoch 2/5
48/48 [==============================] - 308s 6s/step - loss: 0.3923 - accuracy: 0.8472 - val_loss: 0.3359 - val_accuracy: 0.8721
Epoch 3/5
48/48 [==============================] - 310s 7s/step - loss: 0.3158 - accuracy: 0.8844 - val_loss: 0.2853 - val_accuracy: 0.9041
Epoch 4/5
48/48 [==============================] - 304s 6s/step - loss: 0.2809 - accuracy: 0.8994 - val_loss: 0.2641 - val_accuracy: 0.9026
Epoch 5/5
48/48 [==============================] - 276s 6s/step - loss: 0.2787 - accuracy: 0.8955 - val_loss: 0.2586 - val_accuracy: 0.9041


In [31]:
cap = cv2.VideoCapture(0)

In [36]:
while True:
    ret, frame = cap.read()
    #call detection method
    img = cv2.resize(frame,(224,224))
    y_pred = detect_face_mask(img)
    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    if y_pred >0.5:
         draw_label(frame,"No Mask !!!!",(30,30),(0,0,255))
        
    else:
          draw_label(frame,"Mask Detected",(30,30),(0,255,0))
        
    
    print(y_pred)
    
    cv2.imshow("window",frame)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cv2.destroyAllWindows()        
       
    

1/1 [==============================] - 1s 778ms/step
0.99999833
1/1 [==============================] - 0s 283ms/step
0.9999104
1/1 [==============================] - 0s 243ms/step
0.9999145
1/1 [==============================] - 0s 233ms/step
0.99991584
1/1 [==============================] - 0s 233ms/step
0.9999036
1/1 [==============================] - 0s 200ms/step
0.9999411
1/1 [==============================] - 0s 228ms/step
0.9999915
1/1 [==============================] - 0s 214ms/step
0.9999733
1/1 [==============================] - 0s 198ms/step
0.9999527
1/1 [==============================] - 0s 202ms/step
0.99998724
1/1 [==============================] - 0s 199ms/step
0.9999227
1/1 [==============================] - 0s 216ms/step
0.9994244
1/1 [==============================] - 0s 218ms/step
0.9999189
1/1 [==============================] - 0s 215ms/step
0.9999369
1/1 [==============================] - 0s 232ms/step
0.9999785
1/1 [==============================] - 0s 203ms/step

1/1 [==============================] - 0s 183ms/step
0.99999213
1/1 [==============================] - 0s 183ms/step
0.9999966
1/1 [==============================] - 0s 172ms/step
0.99999803
1/1 [==============================] - 0s 219ms/step
0.9999993
1/1 [==============================] - 0s 184ms/step
0.9999985
1/1 [==============================] - 0s 184ms/step
0.99999475
1/1 [==============================] - 0s 183ms/step
0.9999982
1/1 [==============================] - 0s 198ms/step
0.30284858
1/1 [==============================] - 0s 183ms/step
0.061641984
1/1 [==============================] - 0s 201ms/step
1.1634078e-06
1/1 [==============================] - 0s 205ms/step
5.611324e-06
1/1 [==============================] - 0s 194ms/step
1.4354017e-05
1/1 [==============================] - 0s 190ms/step
9.701467e-07
1/1 [==============================] - 0s 221ms/step
1.5283311e-07
1/1 [==============================] - 0s 225ms/step
8.026249e-07
1/1 [=======================

1/1 [==============================] - 0s 219ms/step
0.99996
1/1 [==============================] - 0s 218ms/step
0.9989163
1/1 [==============================] - 0s 200ms/step
0.9998041
1/1 [==============================] - 0s 200ms/step
0.9994277
1/1 [==============================] - 0s 202ms/step
0.9994539
1/1 [==============================] - 0s 235ms/step
0.9994544
1/1 [==============================] - 0s 214ms/step
0.99797034
1/1 [==============================] - 0s 217ms/step
0.99907786
1/1 [==============================] - 0s 213ms/step
0.9989539
1/1 [==============================] - 0s 216ms/step
0.99882585
1/1 [==============================] - 0s 226ms/step
0.99870914
1/1 [==============================] - 0s 231ms/step
0.99947804
1/1 [==============================] - 0s 201ms/step
0.99577445
1/1 [==============================] - 0s 237ms/step
0.9970739
1/1 [==============================] - 0s 199ms/step
0.99626154
1/1 [==============================] - 0s 198ms/st

1/1 [==============================] - 0s 176ms/step
0.999947
1/1 [==============================] - 0s 184ms/step
0.9998759
1/1 [==============================] - 0s 175ms/step
0.9996783
1/1 [==============================] - 0s 189ms/step
0.96931654
1/1 [==============================] - 0s 188ms/step
0.9848941
1/1 [==============================] - 0s 187ms/step
0.9996607
1/1 [==============================] - 0s 175ms/step
0.9913465
1/1 [==============================] - 0s 174ms/step
0.99987924
1/1 [==============================] - 0s 185ms/step
0.9997916
1/1 [==============================] - 0s 197ms/step
0.99988776
1/1 [==============================] - 0s 187ms/step
0.9999559
1/1 [==============================] - 0s 190ms/step
0.99998206
1/1 [==============================] - 0s 183ms/step
0.9999712
1/1 [==============================] - 0s 178ms/step
0.999533
1/1 [==============================] - 0s 175ms/step
0.1572176
1/1 [==============================] - 0s 180ms/step


1/1 [==============================] - 0s 200ms/step
0.99891067
1/1 [==============================] - 0s 201ms/step
0.6374921
1/1 [==============================] - 0s 229ms/step
0.9960091
1/1 [==============================] - 0s 228ms/step
0.94888234
1/1 [==============================] - 0s 230ms/step
0.99740905
1/1 [==============================] - 0s 247ms/step
0.9991573
1/1 [==============================] - 0s 211ms/step
0.9966977
1/1 [==============================] - 0s 214ms/step
0.9966079
1/1 [==============================] - 0s 214ms/step
0.97762007
1/1 [==============================] - 0s 256ms/step
0.99772656
1/1 [==============================] - 0s 198ms/step
0.99094445
1/1 [==============================] - 0s 221ms/step
0.9916928
1/1 [==============================] - 0s 243ms/step
0.9956786
1/1 [==============================] - 0s 197ms/step
0.9975665
1/1 [==============================] - 0s 222ms/step
0.996234
1/1 [==============================] - 0s 215ms/st

1/1 [==============================] - 0s 223ms/step
0.9805209
1/1 [==============================] - 0s 232ms/step
0.9978904
1/1 [==============================] - 0s 233ms/step
0.99845886
1/1 [==============================] - 0s 182ms/step
0.98509264
1/1 [==============================] - 0s 209ms/step
0.9989253
1/1 [==============================] - 0s 245ms/step
0.99601096
1/1 [==============================] - 0s 216ms/step
0.99782205
1/1 [==============================] - 0s 229ms/step
0.9861166
1/1 [==============================] - 0s 214ms/step
0.971442
1/1 [==============================] - 0s 246ms/step
0.98876595
1/1 [==============================] - 0s 199ms/step
0.60239846
1/1 [==============================] - 0s 230ms/step
0.9914764
1/1 [==============================] - 0s 247ms/step
0.870932
1/1 [==============================] - 0s 219ms/step
0.008505236
1/1 [==============================] - 0s 214ms/step
8.193313e-06
1/1 [==============================] - 0s 247m

0.99993247
1/1 [==============================] - 0s 250ms/step
0.9999999
1/1 [==============================] - 0s 239ms/step
0.9999985
1/1 [==============================] - 0s 216ms/step
0.9998645
1/1 [==============================] - 0s 250ms/step
0.99838114
1/1 [==============================] - 0s 234ms/step
0.9977383
1/1 [==============================] - 0s 210ms/step
0.97808117
1/1 [==============================] - 0s 217ms/step
0.99065524
1/1 [==============================] - 0s 217ms/step
0.9950262
1/1 [==============================] - 0s 267ms/step
0.9952281
1/1 [==============================] - 0s 234ms/step
0.9821021
1/1 [==============================] - 0s 220ms/step
0.96398336
1/1 [==============================] - 0s 217ms/step
0.98668194
1/1 [==============================] - 0s 217ms/step
0.95318663
1/1 [==============================] - 0s 234ms/step
0.96676326
1/1 [==============================] - 0s 234ms/step
0.9749126
1/1 [==============================] 

KeyboardInterrupt: 

In [32]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [33]:
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x = pos[0]+text_size[0][0] +2
    end_y = pos[1]+text_size[0][1] +2
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [ ]:
# while True:
#     ret, frame = cap.read()
    
#     coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
#     for x,y,w,h in coods:
#         cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
   
#     cv2.imshow("window",frame)
#     if cv2.waitKey(1) & 0xFF == ord('x'):
#         break

# cv2.destroyAllWindows()        
       
    

In [34]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [35]:
def detect_face(img): 
    coods = haar.detectMultiScale(img)
    return coods

In [37]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_face_mask_detection.h5")
print("Saved model to disk")

Saved model to disk
